## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

And to obtain integer representation, the real value $x_i$

$$ x_i = \sum_{l=-m}^m 2^l q_{i,l}^+ - \sum_{l=-m}^m 2^l q_{i,l}^- $$

To present both position and negetive numbers $q_{i,l}^+$ and $q_{i,l}^-$ are involved.

As before, the domains are given by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q_r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 2
qubits = 4
A = np.array([[3, 1], [-1, 2]])
b = np.array([-1, 5])

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
# print("QM" + str(QM));
for k in range(Dimension): 
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits*2)*i + l
            po2 = (qubits*2)*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2

print("after calculating linear terms: ")
print(QM)

for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits*2)*i + l1
                po2 = (qubits*2)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = (qubits*2)*i + l1 + qubits
                po4 = (qubits*2)*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef

# print("after calculating quadratic terms: ")
# print(QM)

for k in range(Dimension):
    for i in range(Dimension-1):
            for j in range(i+1,Dimension):
                for l1 in range(qubits):
                    for l2 in range(qubits):
                        qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j]
                        po1 = (qubits*2)*i + l1
                        po2 = (qubits*2)*j + l2
                        QM[po1][po2] = QM[po1][po2] + qcef
                        po3 = (qubits*2)*i + l1 + qubits
                        po4 = (qubits*2)*j + l2 + qubits
                        QM[po3][po4] = QM[po3][po4] + qcef
                        po5 = (qubits*2)*i + l1
                        po6 = (qubits*2)*j + l2 + qubits
                        QM[po5][po6] = QM[po5][po6] - qcef
                        po7 = (qubits*2)*i + l1 + qubits
                        po8 = (qubits*2)*j + l2
                        QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)

after calculating linear terms: 
[[ 26.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.  72.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0. 224.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0. 768.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.  -6.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.  96.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0. 512.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -13.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. -16.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   8.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0. 

## Apply QUBO

In [33]:
qbit_list = []
for val in range(len(QM)):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)
Q2 = {}
for i in range(len(QM)):
    for j in range(len(QM)):
        if QM[i][j] != 0:
            Q2[(qbit_list[i],qbit_list[j])] = QM[i][j]
            
print(Q2)
            

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16']
{('q1', 'q1'): 26.0, ('q1', 'q2'): 40.0, ('q1', 'q3'): 80.0, ('q1', 'q4'): 160.0, ('q1', 'q9'): 2.0, ('q1', 'q10'): 4.0, ('q1', 'q11'): 8.0, ('q1', 'q12'): 16.0, ('q1', 'q13'): -2.0, ('q1', 'q14'): -4.0, ('q1', 'q15'): -8.0, ('q1', 'q16'): -16.0, ('q2', 'q2'): 72.0, ('q2', 'q3'): 160.0, ('q2', 'q4'): 320.0, ('q2', 'q9'): 4.0, ('q2', 'q10'): 8.0, ('q2', 'q11'): 16.0, ('q2', 'q12'): 32.0, ('q2', 'q13'): -4.0, ('q2', 'q14'): -8.0, ('q2', 'q15'): -16.0, ('q2', 'q16'): -32.0, ('q3', 'q3'): 224.0, ('q3', 'q4'): 640.0, ('q3', 'q9'): 8.0, ('q3', 'q10'): 16.0, ('q3', 'q11'): 32.0, ('q3', 'q12'): 64.0, ('q3', 'q13'): -8.0, ('q3', 'q14'): -16.0, ('q3', 'q15'): -32.0, ('q3', 'q16'): -64.0, ('q4', 'q4'): 768.0, ('q4', 'q9'): 16.0, ('q4', 'q10'): 32.0, ('q4', 'q11'): 64.0, ('q4', 'q12'): 128.0, ('q4', 'q13'): -16.0, ('q4', 'q14'): -32.0, ('q4', 'q15'): -64.0, ('q4', 'q16'): -128.0, ('q5', 'q5'): -

In [30]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

# print(Q)

sampleset = sampler_auto.sample_qubo(Q2, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0   0   0   0   0  0  0  1  0  0  0  1  0  -26.0     445      0.0
38  0   0   0   0  0  0  1  0  0  0  1  0  -26.0       1 0.083333
1   0   0   0   0  0  0  1  0  0  1  0  0  -21.0      88      0.0
2   0   0   0   0  0  0  1  0  0  1  1  0  -21.0     127      0.0
3   0   0   0   0  0  0  0  1  0  0  1  0  -16.0      71      0.0
4   0   0   0   0  0  0  0  0  0  0  1  0  -16.0      60      0.0
24  0   0   0   0  0  0  0  0  0  0  1  0  -16.0       2 0.083333
5   0   0   0   0  0  0  0  0  0  1  0  0  -13.0       8      0.0
6   0   0   0   0  0  0  0  1  0  1  1  0  -13.0      26      0.0
7   0   0   0   0  0  0  0  1  0  1  0  0   -9.0      10      0.0
8   0   0   0   0  0  0  0  0  0  1  1  0   -9.0      15      0.0
9   0   0   0   0  0  0  1  0  0  0  0  0   -6.0      10      0.0
10  0   0   0   0  0  0  1  0  0  0  0  1   -6.0      18      0.0
11  0   1   0   0  0  0  1  0  0  0  1  0   -1.0      19      0.0
12  0   0 

### Convert Qbit to Decimal 

In [43]:
samples = sampleset.samples()
sol = samples[0]
type(sol)
sol

{'q1': 0, 'q10': 0, 'q11': 0, 'q12': 0, 'q2': 0, 'q3': 0, 'q4': 1, 'q5': 0, 'q6': 0, 'q7': 0, 'q8': 1, 'q9': 0}

In [45]:
samples = sampleset.samples()
type(samples[0])
solution_sample = samples[0]
qbit_per_var_count = int(len(qbit_list)/Dimension)
print(qbit_per_var_count)

# qbit_per_var = []
# for i in range(Dimension):
#     qbit_per_var.append(samples[i,qbit_list[qbit_per_var_count*i:qbit_per_var_count*(i+1)]])

# print(qbit_per_var)
res = []

for i in range(Dimension):
    res.append(0)
    for j in range(qubits):
        pos1 = i*2*qubits+ j
        pos2 = i*2*qubits+ j + qubits
        if qbit_list[pos1] in solution_sample:
            res[i] += (pow(2,j)*solution_sample[qbit_list[pos1]])
        if qbit_list[pos2] in solution_sample:
            res[i] -= (pow(2,j)*solution_sample[qbit_list[pos2]])


# for i in range(Dimension):
#     res.append(0)
#     for j in range(qubits):
#         res[i] += (pow(2,j)*qbit_per_var[i][j] -  pow(2,j)*qbit_per_var[i][qubits+j])

print("Output: ", res)
    


8


ValueError: unknown variable 'q13'

In [5]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q2)
print(J)

({'q1': 23.0, 'q2': 46.0, 'q5': -1.5, 'q6': -3.0, 'q7': 16.5, 'q8': 33.0, 'q3': 7.0, 'q4': 14.0}, {('q1', 'q2'): 10.0, ('q1', 'q5'): 0.5, ('q1', 'q6'): 1.0, ('q1', 'q7'): -0.5, ('q1', 'q8'): -1.0, ('q2', 'q5'): 1.0, ('q2', 'q6'): 2.0, ('q2', 'q7'): -1.0, ('q2', 'q8'): -2.0, ('q3', 'q4'): 10.0, ('q3', 'q5'): -0.5, ('q3', 'q6'): -1.0, ('q3', 'q7'): 0.5, ('q3', 'q8'): 1.0, ('q4', 'q5'): -1.0, ('q4', 'q6'): -2.0, ('q4', 'q7'): 1.0, ('q4', 'q8'): 2.0, ('q5', 'q6'): 5.0, ('q7', 'q8'): 5.0}, 105.0)


In [6]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0 -1 -1 +1 -1 -1 +1 -1 -1 -131.0     741     0.0
8 -1 -1 +1 -1 -1 +1 -1 -1 -131.0       1   0.125
1 -1 -1 +1 -1 +1 -1 -1 -1 -126.0     123     0.0
2 -1 -1 +1 -1 +1 +1 -1 -1 -126.0      92     0.0
3 -1 -1 -1 -1 -1 +1 -1 -1 -121.0       5     0.0
4 -1 -1 -1 +1 -1 +1 -1 -1 -121.0      25     0.0
5 -1 -1 -1 -1 +1 -1 -1 -1 -118.0       2     0.0
6 -1 -1 -1 +1 +1 +1 -1 -1 -118.0      10     0.0
7 -1 -1 -1 -1 +1 +1 -1 -1 -114.0       1     0.0
['SPIN', 9 rows, 1000 samples, 8 variables]
